ERROR: Could not find a version that satisfies the requirement git (from versions: none)
ERROR: No matching distribution found for git


NameError: name 'git' is not defined

In [1]:
import pandas as pd
import numpy as np
from blackboard import Blackboard
from forecast_agent import ForecastAgent
from schedule_agent import ScheduleAgent
from advisor_agent import AdvisorAgent
from orchestrator_agent import OrchestratorAgent
import gradio as gr
from dotenv import load_dotenv
import os
import time

In [2]:
def clean_df(zone):

    #Load CSV data
    hourly_load = pd.read_csv("hrl_load_metered.csv")
    
    #Filter load_area for only PEPCO load zone
    filtered_hourly_load = hourly_load[hourly_load["load_area"].str.contains(zone, case=False)]
    
    #Drop all columns except for datetime, area, and hourly load
    filtered_hourly_load = filtered_hourly_load[["datetime_beginning_ept", "load_area", "mw"]].reset_index(drop = True)
    
    #Parse existing datetime column into new column containing datetime objects
    filtered_hourly_load['datetime'] = pd.to_datetime(filtered_hourly_load['datetime_beginning_ept'],format='%m/%d/%Y %I:%M:%S %p')
    
    #Parse datetime object columns to form new columns
    filtered_hourly_load['hour'] = filtered_hourly_load['datetime'].dt.hour        
    filtered_hourly_load['day'] = filtered_hourly_load['datetime'].dt.day          
    filtered_hourly_load['month'] = filtered_hourly_load['datetime'].dt.month      
    filtered_hourly_load['year'] = filtered_hourly_load['datetime'].dt.year
    
    #Reorder columns
    filtered_hourly_load = filtered_hourly_load[['datetime', 'year', 'month', 'day', 'hour', 'load_area', 'mw']]
    return filtered_hourly_load

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

hourly_load = pd.read_csv("hrl_load_metered.csv")
zones = hourly_load["load_area"].unique().tolist()

#result = orchestrator.run_full_pipeline()
#print(result["output"])

# ---- Helper to format response ----

def run_pipeline(zone):
    # Filter dataframe for the selected zone
    zone_df = clean_df(zone)

    # Initialize blackboard and agents
    blackboard = Blackboard()
    forecast_agent = ForecastAgent(blackboard, zone_df)
    schedule_agent = ScheduleAgent(blackboard)
    advisor_agent = AdvisorAgent(blackboard)

    orchestrator = OrchestratorAgent(
        blackboard=blackboard,
        forecast_agent=forecast_agent,
        schedule_agent=schedule_agent,
        advisor_agent=advisor_agent
    )

    # Accumulate text progressively
    text = f"🚀 Starting full pipeline for zone: **{zone}**"
    yield text

    # Run full pipeline
    result = orchestrator.run_full_pipeline()
    text += "\n\n" + result["output"]
    yield text

    # Stream intermediate steps if available
    if "intermediate_steps" in result and result["intermediate_steps"]:
        text += "\n\n---\n**Tool Execution Log:**\n"
        yield text
        for i, step in enumerate(result["intermediate_steps"], 1):
            action, observation = step
            text += f"\n**Step {i}:** {action.tool}\n```\n{observation}\n```"
            yield text

# ---- Simple Gradio UI ----
custom_css = """
.gradio-container {
    font-family: 'Arial', sans-serif;
    background-color: #eef3fb;
    padding: 20px;
}
.blue-button {
    background-color: #1E90FF;
    color: white;
    font-weight: bold;
}
.output-box {
    border: 2px solid #1E90FF;
    border-radius: 8px;
    padding: 15px;
    background-color: #ffffff;
    overflow-y: auto;
    max-height: 500px;
    white-space: pre-wrap;
}
"""

# ---- Gradio UI with two tabs ----
with gr.Blocks(css=custom_css, title="EV Charging Optimization System") as demo:
    
    with gr.Tabs():

        # ---- Tab 1: EV Charging Optimization ----
        with gr.Tab("⚡ EV Charging Optimization"):
            gr.Markdown("""
# ⚡ EV Charging Optimization System
This project seeks to **find an optimized time to charge your electric vehicle (EV)** based on hourly load forecasts in different zones.  
Select your load zone and click **Generate Report** to get the best charging window recommendations.
""")

            with gr.Row():
                
                with gr.Column(scale=1):
                    zone_dropdown = gr.Dropdown(choices=zones,label="Select Load Zone",value=zones[0])
                    run_btn = gr.Button("🚀 Generate Report", variant="primary", size="lg", elem_classes=["blue-button"])
                    
                with gr.Column(scale=2):
                    output_box = gr.Markdown("Ready to run forecast...", elem_classes=["output-box"])

            run_btn.click(fn=run_pipeline,inputs=[zone_dropdown],outputs=[output_box])

        # ---- Tab 2: System Info ----
        with gr.Tab("ℹ️ System Info"):
            gr.Markdown(
"""
### About This System

**Components:**
- **Forecast Agent**: Trains and uses ML models (Random Forest, XGBoost, Linear Regression) to predict hourly loads
- **Schedule Agent**: Identifies optimal 4-hour charging windows based on minimum load
- **Advisor Agent**: Generates comprehensive report with recommended charging window and forecasted average load
- **Orchestrator Agent**: LangChain-powered coordinator that manages all agents

**Workflow:**
1. Train multiple forecasting models on historical data
2. Evaluate models and select best performer (lowest MAPE)
3. Generate 24-hour load forecast
4. Find optimal 4-hour charging window
5. Generate advisory report with recommendations

**Architecture:**
- Uses blackboard design for inter-agent communication
- LangChain orchestrator with OpenAI Functions for intelligent task routing
- Natural language interface for easy interaction
"""
            )

# ---- Launch ----
demo.launch(
    server_name="0.0.0.0",
    server_port=7911
)

* Running on local URL:  http://0.0.0.0:7911
* To create a public link, set `share=True` in `launch()`.
